# Save and retrieve jobs

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.]( https://quantum.cloud.ibm.com/docs/guides/save-jobs)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/save-jobs)
</Admonition>
</CloudContent>

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit-ibm-runtime~=0.37.0
```
</details>

Quantum workflows often take a while to complete and can run over many sessions. Restarting your Python kernel means you'll lose any results stored in memory. To avoid loss of data, you can save results to file and retrieve results of past jobs from IBM Quantum so your next session can continue where you left off.

## Retrieve jobs from IBM Quantum

<LegacyContent>
IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs on the [Workloads page](https://quantum.ibm.com/workloads).
</LegacyContent>
<CloudContent>
IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs on the [Workloads page](https://quantum.cloud.ibm.com/workloads).
</CloudContent>
To find a job programmatically, use the [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit-runtime-service#jobs) method. By default, this returns the most recent jobs, but you can also filter jobs by backend name, creation date, and more. The following cell finds any jobs submitted in the last three months. The `created_after` argument must be a [`datetime.datetime`](https://docs.python.org/3.8/library/datetime.html#datetime.datetime) object.

In [1]:
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)

service = QiskitRuntimeService()
jobs_in_last_three_months = service.jobs(created_after=three_months_ago)
jobs_in_last_three_months[:3]  # show first three jobs

[<RuntimeJobV2('cznay3gqw630008tn290', 'sampler')>,
 <RuntimeJobV2('cznay1rp3xeg008heh5g', 'sampler')>,
 <RuntimeJobV2('cznaxzqtj3e0008k0910', 'sampler')>]

You can also select by backend, job state, session, and more. For more information, see [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit-runtime-service#jobs) in the API documentation.

Once you have the job ID, use the [`QiskitRuntimeService.job`](/api/qiskit-ibm-runtime/qiskit-runtime-service#job) method to retrieve it.

In [2]:
# Get ID of most recent successful job for demonstration.
# This will not work if you've never successfully run a job.
successful_job = next(
    j for j in service.jobs(limit=1000) if j.status() == "DONE"
)
job_id = successful_job.job_id()
print(job_id)

cznay3gqw630008tn290


In [3]:
retrieved_job = service.job(job_id)
retrieved_job.result()

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=16>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-03-31 15:04:50', stop='2025-03-31 15:04:51', size=100>)])}, 'version': 2})

<CodeAssistantAdmonition
  tagLine="Always forgetting how to retrieve a job? Try this prompt with Qiskit Code Assistant:"
  prompts={[
    "# Retrieve job JOB_ID from IBM Runtime and print the result"
  ]}
/>

## Save results to disk

You may also want to save results to disk. To do this, use Python's built-in JSON library with Qiskit Runtime's encoders.

In [4]:
import json
from qiskit_ibm_runtime import RuntimeEncoder

with open("result.json", "w") as file:
    json.dump(retrieved_job.result(), file, cls=RuntimeEncoder)

You can then load this array from disk in a separate kernel.

In [5]:
from qiskit_ibm_runtime import RuntimeDecoder

with open("result.json", "r") as file:
    result = json.load(file, cls=RuntimeDecoder)

result

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=100, num_bits=16>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-03-31 15:04:50', stop='2025-03-31 15:04:51', size=100>)])}, 'version': 2})